<a href="https://colab.research.google.com/github/gabriellcortes/ieeecisunb/blob/main/3_periodo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import random
from scipy.stats import mode


df = pd.read_csv('/content/drive/MyDrive/ieee cis/winequality.csv')
df = df.loc[:, df.columns != 'Unnamed: 0']


In [ ]:
df.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,wine_is_red
count,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000
mean,7.215307,0.339666,0.318633,5.443235,0.056034,30.525319,115.744574,0.994697,3.218501,0.531268,10.491801,5.818378,0.246114
std,1.296434,0.164636,0.145318,4.757804,0.035034,17.749400,56.521855,0.002999,0.160787,0.148806,1.192712,0.873255,0.430779
min,3.800000,0.080000,0.000000,0.600000,0.009000,1.000000,6.000000,0.987110,2.720000,0.220000,8.000000,3.000000,0.000000
25%,6.400000,0.230000,0.250000,1.800000,0.038000,17.000000,77.000000,0.992340,3.110000,0.430000,9.500000,5.000000,0.000000
50%,7.000000,0.290000,0.310000,3.000000,0.047000,29.000000,118.000000,0.994890,3.210000,0.510000,10.300000,6.000000,0.000000
75%,7.700000,0.400000,0.390000,8.100000,0.065000,41.000000,156.000000,0.996990,3.320000,0.600000,11.300000,6.000000,0.000000
max,15.900000,1.580000,1.660000,65.800000,0.611000,289.000000,440.000000,1.038980,4.010000,2.000000,14.900000,9.000000,1.000000


In [ ]:
df.head()

,Unnamed: 0,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,wine_is_red
0,0,5.9,0.61,0.08,2.1,0.071,16.0,24.0,0.99376,3.56,0.77,11.1,6,1
1,1,8.2,1.00,0.09,2.3,0.065,7.0,37.0,0.99685,3.32,0.55,9.0,6,1
2,2,6.4,0.24,0.49,5.8,0.053,25.0,120.0,0.99420,3.01,0.98,10.5,6,0
3,3,6.6,0.23,0.26,1.3,0.045,16.0,128.0,0.99340,3.36,0.60,10.0,6,0
4,4,7.2,0.24,0.29,3.0,0.036,17.0,117.0,0.99411,3.36,0.68,10.1,6,0


In [ ]:
df['wine_is_red'] = df['wine_is_red'].replace({1:'wine_is_red', 0:'wine_is_not_red'})

df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,wine_is_red
0,5.9,0.61,0.08,2.1,0.071,16.0,24.0,0.99376,3.56,0.77,11.1,6,wine_is_red
1,8.2,1.00,0.09,2.3,0.065,7.0,37.0,0.99685,3.32,0.55,9.0,6,wine_is_red
2,6.4,0.24,0.49,5.8,0.053,25.0,120.0,0.99420,3.01,0.98,10.5,6,wine_is_not_red
3,6.6,0.23,0.26,1.3,0.045,16.0,128.0,0.99340,3.36,0.60,10.0,6,wine_is_not_red
4,7.2,0.24,0.29,3.0,0.036,17.0,117.0,0.99411,3.36,0.68,10.1,6,wine_is_not_red


In [ ]:
def criar_treino_teste(data,prop):
  shuffled_index = np.random.permutation(df.index)
  rand_df = df.reindex(shuffled_index)
  # Divide o numero de linhas para treino e teste
  last_train_row = int((1-prop)*len(df))  
  train_df = rand_df.iloc[0:last_train_row]
  test_df = rand_df.iloc[last_train_row:]
  return train_df,test_df


In [ ]:
def normalizacao(data):
  data = data.to_numpy() # transformo em numpy
  media = data.mean(axis=0)
  desvio = data.std(axis=0)
  data = (data - media)/desvio #normalizando
  return data

In [ ]:
def matrix_confusa(Predicted, Class):
  df_matrix_confusa = pd.DataFrame({'Class':Class.reshape(-1),
                          'Predicted':Predicted.reshape(-1)}) 
  matrix = pd.crosstab(
      df_matrix_confusa['Class'], df_matrix_confusa['Predicted'])
  return matrix

In [ ]:
def accuracy(matrix_confusa):
  valor = (np.diagonal(matrix)).sum() / matrix.to_numpy().sum()
  return valor

In [ ]:
def knn_algorithm(train_x,test_x, train_y, k):
  train_x = normalizacao(train_x)
  test_x = normalizacao(test_x)

  results = np.array([])

  
  for vetor_teste in test_x: #Para cada valor de teste faça:
    sub = (vetor_teste - train_x)**2 # 1- Calculo da distancia euclediana para todos os pontos de treino
    soma = sub.sum(axis=1) # 2
    distancia = np.sqrt(soma) # 3 - Distancia calculada
    idx = np.argpartition(distancia, k)[:k] # Capturando os indices dos k mais proximos
    amostra = train_y[idx] # Vendo a classificação desses pontos na variavel resposta
    resp = mode(amostra)[0][0] # Coletando a moda dos pontos mais proximos
    results = np.append(results, resp, axis=0) # adicionando aos resultados
  return results
    

In [ ]:
train, test = criar_treino_teste(df,0.25) # Criando df de treino e de teste

In [ ]:
lista = list(df) # Nome das colunas
lista.remove('wine_is_red') # Removendo a variavel resposta

train_x = train[lista]
test_x = test[lista]
train_y = train[["wine_is_red"]].to_numpy()
test_y = test[["wine_is_red"]].to_numpy()

In [ ]:
init = time()
result = knn_algorithm(train_x,test_x, train_y, 3)
end = time()
end-init

0.7880113124847412

In [ ]:
matrix = matrix_confusa(test_y, result)
matrix

Predicted,wine_is_not_red,wine_is_red
Class,,
wine_is_not_red,1233,6
wine_is_red,5,381


In [ ]:
accuracy(matrix)

0.9932307692307693

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

#Create KNN Classifier
knn = KNeighborsClassifier(n_neighbors=3)

#Train the model using the training sets
knn.fit(train_x, train_y)

#Predict the response for test dataset
y_pred = knn.predict(test_x)

In [ ]:
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(test_y, y_pred))

Accuracy: 0.9501538461538461
